In [2]:
import json
import pymysql
from pymysql.cursors import DictCursor

# 連接到 MySQL 資料庫
def connect_to_db():
    connection = pymysql.connect(
        host='localhost',
        port=3306,
        user='root',
        password='P@ssw0rd',
        db='member',
        charset='utf8mb4',
        cursorclass=pymysql.cursors.DictCursor
    )
    return connection

# 創建 members 表
def create_table():
    connection = connect_to_db()
    try:
        with connection.cursor() as cursor:
            create_table_sql = """
            CREATE TABLE IF NOT EXISTS members (
                id INT AUTO_INCREMENT PRIMARY KEY,
                member_name VARCHAR(255) NOT NULL,
                member_gender VARCHAR(10),
                member_phone VARCHAR(20),
                member_age INT,
                member_city VARCHAR(50),
                encoding TEXT NOT NULL
            );
            """
            cursor.execute(create_table_sql)
        connection.commit()
    finally:
        connection.close()

# 從 JSON 文件中加載會員數據
def load_members_from_json(file_path='members.json'):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# 將會員數據插入到資料庫中
def insert_members_to_db(members_data):
    connection = connect_to_db()
    try:
        with connection.cursor() as cursor:
            for member in members_data['members']:
                sql = """
                INSERT INTO members (member_name, member_gender, member_phone, member_age, member_city, encoding)
                VALUES (%s, %s, %s, %s, %s, %s)
                """
                member_age = member.get('member_age', 0)
                if isinstance(member_age, str) and not member_age.isdigit():
                    member_age = 0
                else:
                    member_age = int(member_age)
                    
                cursor.execute(sql, (
                    member.get('member_name', ''),
                    member.get('member_gender', ''),
                    member.get('member_phone', ''),
                    member_age,
                    member.get('member_city', ''),
                    json.dumps(member['encoding'])  # 將 encoding 轉換為 JSON 字符串                   
                ))
        connection.commit()
    finally:
        connection.close()

# 主函數
def main():
    # 創建表
    create_table()
    print("資料表已成功創建或已存在。")
    
    # 加載並插入會員數據
    members_data = load_members_from_json()
    insert_members_to_db(members_data)
    print("會員數據已成功存入 MySQL 資料庫。")

if __name__ == "__main__":
    main()

資料表已成功創建或已存在。
會員數據已成功存入 MySQL 資料庫。
